<a href="https://colab.research.google.com/github/nowshaba/NLTK/blob/main/NLTKchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import nltk # natural language tool kits
import numpy
import tensorflow as tf
import json
import os

In [ ]:

def load_data(json_file='/starwarsintents.json') :
    import json
    with open(json_file) as f:
        dataset = json.load(f)
    
    return dataset
#daa = load_data()
#for i in daa:
  #print(i['tag'])
  #for r in i['responses']:
     # print(r)
  
    
      
  
  

   


In [ ]:

def tokenize_words(dataset) :        
    words = []
    intents_x = []
    for intent in dataset:
        for p in intent['patterns']+intent['responses'] : # consider both Q & A
            #tokens = nltk.word_tokenize(p)
            tokens = nltk.regexp_tokenize(p, "[\w']+") # can remove ?
            words.extend(tokens)
            intents_x.append(tokens)

    intent_tags = []
    intents_y = []
    for intent in dataset:
        for p in intent['patterns']+intent['responses'] : # consider both Q & A
            intents_y.append(intent["tag"])
        if intent['tag'] not in intent_tags:
            intent_tags.append(intent['tag'])
            
    intent_tags = sorted(intent_tags)

    #print("\nwords=")        
    #print(words)   
    #print("\nintents_x=")  
    #print(intents_x) 
    #print("\nintents_y=")   
    print(intents_y)
    #print("\nintent_tags=")   
    print(intent_tags)

    return (words, intents_x, intents_y, intent_tags)
dataset = load_data()
t=tokenize_words(dataset)


['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'alive', 'alive', 'alive', 'alive', 'alive', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl'

In [ ]:
def stemming(words) :
    
    
    stemmer = nltk.stem.LancasterStemmer()
    words_tmp = []
    for w in words :
        #if w != '?' : words_tmp.append(stemmer.stem(w.lower()))
        words_tmp.append(stemmer.stem(w.lower()))
            
    root_words = sorted(set(words_tmp)) # set() will remove duplications

    #print("\nAfter stemming, root words=") 
    #print(root_words)    
    
    return root_words
#d=load_data() 
#words, intents_x, intents_y, intent_tag =tokenize_words(d)
#s=stemming(words)    

In [ ]:
def get_train_data(root_words, intents_x, intents_y, intent_tags) :

    stemmer = nltk.stem.LancasterStemmer()

    training = []
    output = []

    for i in range(len(intents_x)):
        #for x 
        bag_of_words = []
        words_tmp = []
        #print(intents_x[i])
        for w in intents_x[i] :
            #print(w)
            words_tmp.append(stemmer.stem(w.lower()))        
            
        for w in root_words:
            found = 1 if w in words_tmp else 0
            bag_of_words.append(found)
            
        training.append(bag_of_words)
        #print(bag_of_words)

        #for y
        output_row = [0] * len(intent_tags)
        #print(output_row)
        #print(output_row[intent_tags.index(intents_y[i])])
        output_row[intent_tags.index(intents_y[i])] = 1
        
        output.append(output_row)
        #print(output)

    train_x = numpy.array(training)
    train_y = numpy.array(output)   
    
    print("\n")
    print(train_x.shape)
    print(train_x)
    print(train_y.shape)
    print(train_y)
    
    return (train_x, train_y)
#d=load_data() 
#words, intents_x, intents_y, intent_tag =tokenize_words(d)
#s=stemming(words)   
#f= get_train_data(s, intents_x, intents_y, intent_tag)

In [ ]:
def build_model(train_x, train_y) :
    epochs = 1000
    batch_size = 32  

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(batch_size, input_shape=(len(train_x[0]),))) # input layer
    model.add(tf.keras.layers.Dense(batch_size)) # hidden layer 1
    model.add(tf.keras.layers.Dense(batch_size)) # hidden layer 2
    model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax')) # output layer

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate accuracy of the model '''
    model.summary() # for debugging
    test_loss, test_acc = model.evaluate(train_x, train_y, verbose=0)
    print("\nTested Loss:{} Acc:{}".format(test_loss, test_acc))
    
    return model
#d=load_data() 
#words, intents_x, intents_y, intent_tag =tokenize_words(d)
#s=stemming(words)   
#f, r= get_train_data(s, intents_x, intents_y, intent_tag)
#m = build_model(f, r)  


['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'alive', 'alive', 'alive', 'alive', 'alive', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl'

In [ ]:
def prepare_model(dataset, model_file="model.chatbot") :
    ''' tokenize '''
    words, intents_x, intents_y, intent_tags = tokenize_words(dataset)
    
    ''' stemming '''
    root_words = stemming(words)

    '''if os.path.isdir(model_file) or os.path.isfile(model_file):
        Load existing model 
        model = tf.keras.models.load_model(model_file)
        print("Loading model successfully")
    else :'''
        #''' Create a new model '''                 
    train_x, train_y = get_train_data(root_words, intents_x, intents_y, intent_tags)
    model = build_model(train_x, train_y)
        #model.save(model_file)
    
    return (model, root_words, intent_tags)

# New Section

In [ ]:
def chat(model, root_words, intent_tags, dataset):
    print("Start talking with the bot (type quit to stop)")
    while True:
        question = input("You: ")
        if question.lower() == "quit": break
            
        print(root_words) 

        ''' tokenize '''
        tokens = nltk.regexp_tokenize(question, "[\w']+")
        print(tokens)

        ''' stemming '''
        stemmer = nltk.stem.LancasterStemmer()
        tokens_tmp = []
        for t in tokens:
            tokens_tmp.append(stemmer.stem(t.lower()))
        print(tokens_tmp)
        
        ''' bag of words '''
        bag_of_words = []
        for w in root_words:
            
            if w in tokens_tmp :
                found = 1
                print(w)
            else :
                found = 0
            bag_of_words.append(found)
        print(bag_of_words)

        ''' model predict to find a tag '''
        probilities = model.predict(numpy.array([bag_of_words]))
        max_index = numpy.argmax(probilities)
        max_probility = probilities[0][max_index]
        if max_probility < 0.8 :
            print("Sorry, I don't know what you mean. (confidence: {})".format(max_probility))
            continue
        else :
            found_tag = intent_tags[max_index]

        ''' randomly choose a response according to the found tag '''
        for intent in dataset:
            if intent['tag'] == found_tag: 
                print("{} (confidence: {})".format(numpy.random.choice(intent['responses']), max_probility))


'''
Start up
'''
dataset = load_data()
model, root_words, intent_tags = prepare_model(dataset)
chat(model, root_words, intent_tags, dataset)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'tasks', 'alive', 'alive', 'alive', 'alive', 'alive', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'Menu', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl', 'hepl'